In [1]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import os
from glob import glob
from ipywidgets import interact
import matplotlib.patches as patches
import seaborn as sns
import ipywidgets as widgets

In [2]:
annFile = '/opt/ml/detection/dataset/train.json'
coco = COCO(annFile)
cats = coco.loadCats(coco.getCatIds())
nms = [cat['name'] for cat in cats]
img_id = coco.getImgIds()
img_info = coco.loadImgs(img_id)
fnames = [info['file_name'] for info in img_info]


loading annotations into memory...
Done (t=0.20s)
creating index...
index created!


In [5]:
button = widgets.Button(description='save image')
text = widgets.Text()
output = widgets.Output()
palette = sns.color_palette('bright')
img_save_dir = '/opt/ml/wrong_img'
os.makedirs(img_save_dir, exist_ok=True)

@interact(idx=(0, len(fnames)-1), cls_id=(0, 100), double_click=False)
def showImg(idx, cls_id, double_click):
    fig, ax = plt.subplots(1, 1, dpi=200)
    img = io.imread(os.path.join('/opt/ml/detection/dataset', fnames[idx]))
    annIds = coco.getAnnIds(imgIds=idx)
    anns = coco.loadAnns(annIds)
    
    ax.imshow(img)
    for ann in anns:
        ann_id = ann['id']
        class_idx = ann['category_id']
        ax.set_title(str(ann['image_id'])+'.jpg', fontsize=5)
        ax.set_xticks([])
        ax.set_yticks([])
        for pos in ['right', 'top', 'bottom', 'left']:
            ax.spines[pos].set_visible(False)
        if(class_idx == cls_id or cls_id==10):
            x,y,w,h = ann['bbox']
            color = palette[class_idx]
            ax.add_patch(
                patches.Rectangle(
                    (x,y), w, h,
                    edgecolor=color,
                    fill=False,
                    ),
                )
            text_y = y-15 if y>15 else y+15 
            plt_text = ax.text(x+8,text_y, f'{nms[class_idx]}_{ann_id}', color='white', fontsize='3', weight='semibold', backgroundcolor=color)
            plt_text.set_bbox(dict(
                facecolor=palette[class_idx],  # background color
                alpha=0.6,  # background alpha
                edgecolor='none',  # border color
                pad=2
            ))
    if double_click:
        fig.savefig(os.path.join(img_save_dir, str(ann['image_id'])+'_'+text.value))
        text.value = ''
    display(text, output)

interactive(children=(IntSlider(value=2441, description='idx', max=4882), IntSlider(value=50, description='cls…